---
date:
  created: 2025-03-22
  updated: 2025-03-30

categories:
- Data preparation

tags:
- Polars
- Hugging Face

slug: clean-uci-drug-review-dataset
---

# Cleaning UCI ML Drug Review Dataset

This is my [:simple-polars: Polars][1]-centric adaptation of the tutorial
[:simple-huggingface: Hugging Face NLP Course: Time to slice and dice][2],
which cleans the UCI ML Drug Review dataset available on
[:fontawesome-brands-kaggle: Kaggle][3].

Cleaned dataset on Hugging Face Hub: [:material-database: dd-n-kk/uci-drug-review-cleaned][4]

<a href="https://colab.research.google.com/github/dd-n-kk/notebooks/blob/main/blog/clean-uci-drug-review-dataset.ipynb" target="_parent">
    :simple-googlecolab: Colab notebook
</a>

<!-- more -->

## Preparations

In [ ]:
# Set this to an empty string to avoid saving the dataset file.
DATA_DIR = "uci-drug-review-cleaned/"

# Set these to empty strings to avoid uploading the dataset.
REPO_ID = "dd-n-kk/uci-drug-review-cleaned"
SECRET = "HF_TOKEN"

In [ ]:
!uv pip install --system -Uq polars

In [ ]:
import kagglehub
import polars as pl
from polars import col

In [ ]:
path = kagglehub.dataset_download("jessicali9530/kuc-hackathon-winter-2018")
!mkdir data/ && cp -t data/ {path}/* && ls -hAil data

100%|██████████| 40.7M/40.7M [00:00<00:00, 44.6MB/s]

Extracting files...


total 106M
2364654 -rw-r--r-- 1 root root 27M Mar 22 13:17 drugsComTest_raw.csv
2364655 -rw-r--r-- 1 root root 80M Mar 22 13:17 drugsComTrain_raw.csv


In [ ]:
SEED = 777
pl.set_random_seed(SEED)

# Configure Polars for more complete display.
_ = pl.Config(
    tbl_cols=-1,
    tbl_rows=100,
    tbl_width_chars=-1,
    float_precision=3,
    fmt_str_lengths=500,
    fmt_table_cell_list_len=-1,
)

In [ ]:
df = pl.read_csv("data/drugsComTrain_raw.csv")

## Observations

- `drugName`:
    - Capitalization is inconsistent ("A + D Cracked Skin Relief" vs. "femhrt").
    - There are combination prescriptions ("Ethinyl estradiol / norgestimate").

- `condition`:
    - There are 899 nulls.
    - There are corrupted values:
        - "0`</span>` users found this comment helpful."
        - "zen Shoulde" should probably be "Frozen Shoulder".

- `review`:
    - There are HTML entities (`&#039;`).
    - There are emojis (❤️❤️❤️).

- `date`:
    - The format could be simplified ("1-Apr-08").

In [ ]:
df.describe()

statistic,uniqueID,drugName,condition,review,rating,date,usefulCount
str,f64,str,str,str,f64,str,f64
"""count""",161297.000,"""161297""","""160398""","""161297""",161297.000,"""161297""",161297.000
"""null_count""",0.000,"""0""","""899""","""0""",0.000,"""0""",0.000
"""mean""",115923.585,null,null,null,6.994,null,28.005
"""std""",67004.445,null,null,null,3.272,null,36.404
"""min""",2.000,"""A + D Cracked Skin Relief""","""0</span> users found this comment helpful.""",""""" please tell the ones who is suffering from anxiety to use lavender chamomile spray by air wick. it gives immediate relief , doctors not letting know patients about this. please spread the word!!. Please keep this post here.""""",1.000,"""1-Apr-08""",0.000
"""25%""",58063.000,null,null,null,5.000,null,6.000
"""50%""",115744.000,null,null,null,8.000,null,16.000
"""75%""",173776.000,null,null,null,10.000,null,36.000
"""max""",232291.000,"""femhrt""","""zen Shoulde""","""""❤️❤️❤️ Cialis for US!! I wish I had my husband start this decades ago """"",10.000,"""9-Sep-17""",1291.000


In [ ]:
df.sample(10, seed=SEED)

uniqueID,drugName,condition,review,rating,date,usefulCount
i64,str,str,str,i64,str,i64
63419,"""Epiduo""","""Acne""","""""I have been using epiduo for almost a week now and it has been burning a little bit, so i cut back on how much i put on my face and it feels much better. I did not have alot of acne at all before using it but only a few on my cheeks and those are all gone now after a few days of use. The few bumps on ny forehead are still there but im positive the epiduo is fighting to get rid of it. I will give it a couple of weeks :). (avoid using in eye area, and area right AROUND nose... those are sensitive…",8,"""27-Jun-16""",4
152744,"""Doxycycline""","""Bacterial Infection""","""""Have been taking this med now for the past 8 days, for an acute sinusitis infection, and paid strict attention to the pharmacists instructions..... YOU MUST DRINK A FULL GLASS OF WATER after you&#039;ve taken it. Don&#039;t disregard these rules, or you will have severe side effects. Otherwise I&#039;m hoping this time, doxy will clear it up as I was on amoxy and that didn&#039;t. This infection keeps repeating because of two bouts of pneumonia, and I&#039;m sure residual infection wasn&#039…",8,"""3-Nov-17""",2
126485,"""Brimonidine""","""Rosacea""","""""Stay away from this medication. The 1st day used, I was impressed, but 2 days later I had a rebound and my face was burning like hell. I waited like 4 days to re-apply and see what happens, but I lost sensation on my lips and they got swollen. Then, after 24 hours the rebound got worse and was like two weeks after my skin stopped burning. I still don&#039;t understand how this product was approved by the FDA.""""",1,"""27-Jun-16""",21
208405,"""Oseltamivir""","""Influenza""","""""Sick sick with flu, tamiflu added to illness, nausea &amp; vomiting, don&#039;t know if made flu last shorter, made me too nauseous to know, just ride out the virus, plan on being sick for a week, don&#039;t burden your body with meds that may make you sick""""",2,"""25-Dec-15""",17
166105,"""Levonorgestrel""",null,"""""Well I&#039;m the first one reviewing this... I got my Kyleena IUD placed on Dec. 2nd 2016. It was painful but no more so than other IUDs I&#039;ve read about. The issues I&#039;m having is contant spotting. I&#039;ve been bleeding lightly (where I have to wear a light pad so I don&#039;t ruin my underwear every day) for two months now. The office told me about a month ago that my spotting is normal and most likely means once I stop spotting I&#039;ll be done with my period for the five years K…",5,"""30-Jan-17""",17
16075,"""Ethinyl estradiol / norethindrone""","""Birth Control""","""""I&#039;m 14 years old and I&#039;ve been on it for 4 months and I hate it. I&#039;m on it for regular periods since my blood flows too quickly and it has done nothing. I&#039;ve been on my period for 4 months, light mostly but heavy sometimes. I also went on it for my acne but it did nothing. I gained 4 pounds and now I have stubborn belly fat. I also have huge stretch marks (idk if its related but they showed up when I took it). Also, I have no motivation anymore. Don&#039;t take this please!""""",3,"""27-May-16""",4
223196,"""Estradiol""","""Postmenopausal Symptoms""","""""I have had a great experience with the Minivelle patch. I was suffering with many, many, hot flashes a day and night sweats that left me drenched! I am down to a handful of flashes a day and the night sweats are gone!""""",10,"""15-Jul-13""",41
102766,"""Aripiprazole""","""Major Depressive Disorde""","""""Weight gain and fatigue.""""",3,"""29-Jul-14""",15
85534,"""Ethinyl estradiol / norgestimate""","""Birth Control""","""""This Is An Amazing Birth Control. I&#039;ve Been On This For Five Years And Counting. I Never Got Pregnant, My Menstrual Last About 3-4 Days(Regular Period ) ,No Cramps, Face Is Clear Of Acne, From Time To Time Ive Gotten Moody But Overall I Love It! I&#039;m Looking Forward To Getting Off And Starting A Family.""""",9,"""17-

## Verifying uniqueness of `uniqueID`

Unlike the version used in the Hugging Face tutorial,
the Kaggle-hosted dataset does have a name for the first column.
So here we just verify that each row indeed has a unique ID.

In [ ]:
assert df.get_column("uniqueID").n_unique() == len(df)

## Scanning for invalid `drugName`s

We scan for invalid drug names by looking for unusual characters.
Fortunately most if not all drug names seem to be valid.

In [ ]:
(
    df.filter(col("drugName").str.contains(r"[^[:alnum:] '(),./-]"))
    .get_column("drugName")
    .value_counts()
    .sort("count", descending=True)
)

drugName,count
str,u32
"""Tylenol with Codeine #3""",65
"""Coricidin HBP Cold & Flu""",5
"""Tylenol with Codeine #4""",4
"""Mucinex Fast-Max Severe Congestion & Cough""",2
"""Dimetapp Children's Cold & Cough""",2
"""Aleve-D Sinus & Cold""",1
"""Coricidin HBP Cough & Cold""",1
"""Sudafed PE Pressure + Pain""",1
"""A + D Cracked Skin Relief""",1


## Handling capitalization of `drugName`s

There are very few lowercase drug names and all of them are valid.
We can also confirm that no drug name is cased differently in the dataset.
To preserve the original data as much as possible,
I title-case the lowercase names instead of lowercasing all drug names.

In [ ]:
(
    df.filter(col("drugName").str.contains(r"^[^A-Z]"))
    .get_column("drugName")
    .value_counts()
    .sort("count", descending=True)
)

drugName,count
str,u32
"""ella""",51
"""femhrt""",3
"""depo-subQ provera 104""",1


In [ ]:
assert (
    df.get_column("drugName").n_unique()
    == df.get_column("drugName").str.to_lowercase().n_unique()
)

In [ ]:
replacements = {
    "ella": "Ella",
    "femhrt": "Femhrt",
    "depo-subQ provera 104": "Depo-SubQ Provera 104"
}

df = df.with_columns(col("drugName").replace(replacements))

## Cleaning invalid `condition`s

The search for unusual characters yields a group of rows with invalid `condition`s:
"??`</span>` users found this comment helpful." I decide to replace them with nulls.

In [ ]:
q = df.filter(col("condition").str.contains(r"[^[:alnum:] '(),./-]")); len(q)

900

In [ ]:
q.select(pl.exclude("review")).sample(10, seed=SEED)

uniqueID,drugName,condition,rating,date,usefulCount
i64,str,str,i64,str,i64
81207,"""Yaz""","""0</span> users found this comment helpful.""",1,"""11-Feb-17""",0
126293,"""Viibryd""","""15</span> users found this comment helpful.""",5,"""14-Oct-11""",15
24826,"""Deplin""","""13</span> users found this comment helpful.""",6,"""9-Aug-16""",13
67383,"""Provera""","""4</span> users found this comment helpful.""",1,"""27-Mar-16""",4
212726,"""Phenergan""","""3</span> users found this comment helpful.""",10,"""3-Jan-09""",3
220738,"""Loestrin 24 Fe""","""9</span> users found this comment helpful.""",5,"""13-Aug-10""",9
33084,"""Seasonique""","""2</span> users found this comment helpful.""",8,"""29-Jun-10""",2
159396,"""Estrace Vaginal Cream""","""64</span> users found this comment helpful.""",8,"""14-Sep-11""",64
209258,"""Geodon""","""5</span> users found this comment helpful.""",6,"""18-Feb-09""",5


In [ ]:
df = df.with_columns(
    pl.when(col("condition").str.contains("</span>", literal=True))
    .then(pl.lit(None))
    .otherwise(col("condition"))
    .alias("condition")
)

## Correcting corrupted `condition`s

The search for lowercase condition names leads to a few unexpected findings:

- Some `condition` fields mistakenly record drug names instead of condition names.
- Quite a few condition names lost their prefixes, suffixes, or both.
  Indeed, seemingly all leading `F`s and terminating `r`s were cut off.

I decide to handle the corrupted entries one by one:
Correct those I can recognize and nullify those I cannot.

In [ ]:
(
    df.filter(col("condition").str.contains(r"^[^A-Z]"))
    .get_column("condition")
    .value_counts()
    .sort("count", descending=True)
)

condition,count
str,u32
"""ibromyalgia""",1791
"""mance Anxiety""",187
"""moterol)""",79
"""emale Infertility""",65
"""atigue""",53
"""min)""",45
"""ge (amlodipine / valsartan)""",33
"""acial Wrinkles""",32
"""moterol / mometasone)""",29


In [ ]:
replacements = {
    "ibromyalgia": "Fibromyalgia",
    "mance Anxiety": "Performance Anxiety",
    "emale Infertility": "Female Infertility",
    "atigue": "Fatigue",
    "acial Wrinkles": "Facial Wrinkles",
    "zen Shoulde": "Frozen Shoulder",
    "eve": "Fever",
    "lic Acid Deficiency": "Folic Acid Deficiency",
    "von Willebrand's Disease": "Von Willebrand's Disease",
    "amilial Mediterranean Feve": "Familial Mediterranean Fever",
    "cal Segmental Glomerulosclerosis": "Focal Segmental Glomerulosclerosis",
    "ailure to Thrive": "Failure to Thrive",
    "amilial Cold Autoinflammatory Syndrome": "Familial Cold Autoinflammatory Syndrome",
    "actor IX Deficiency": "Factor IX Deficiency",
    "acial Lipoatrophy": "Facial Lipoatrophy",
    "ungal Pneumonia": "Fungal Pneumonia",
    "llicular Lymphoma": "Follicular Lymphoma",
    "unctional Gastric Disorde": "Functional Gastric Disorder",
    "ungal Infection Prophylaxis": "Fungal Infection Prophylaxis",
    "ibrocystic Breast Disease": "Fibrocystic Breast Disease",
    "llicle Stimulation": "Follicle Stimulation",

    "moterol)": None,
    "min)": None,
    "ge (amlodipine / valsartan)": None,
    "moterol / mometasone)": None,
    "min / sitagliptin)": None,
    "mulation) (phenylephrine)": None,
    "min / saxagliptin)": None,
    "mis": None,
    "min / pioglitazone)": None,
    "ge HCT (amlodipine / hydrochlorothiazide / valsartan)": None,
    "min / rosiglitazone)": None,
    "tic (mycophenolic acid)": None,
    "t Care": None,
    "t Pac with Cyclobenzaprine (cyclobenzaprine)": None,
    "me": None,
    "mist (": None,
    "m Pain Disorde": None,
}

df = df.with_columns(col("condition").replace(replacements))

Identifying all conditions losing their terminating `r`s would require nontrivial
domain knowledge and effort.
I can only come up with and correct "disorde" and "(f)eve",
which should be "disorder" and "fever", respectively.

In [ ]:
(
    df.filter(col("condition").str.ends_with("eve"))
    .get_column("condition")
    .value_counts()
)

condition,count
str,u32
"""Q Feve""",1
"""Typhoid Feve""",2
"""Pain/Feve""",13


In [ ]:
(
    df.filter(col("condition").str.ends_with("rde"))
    .get_column("condition")
    .value_counts()
)

condition,count
str,u32
"""Binge Eating Disorde""",72
"""Temporomandibular Joint Disorde""",30
"""Bleeding Disorde""",7
"""Social Anxiety Disorde""",389
"""Borderline Personality Disorde""",151
"""Hypoactive Sexual Desire Disorde""",7
"""Shift Work Sleep Disorde""",60
"""Post Traumatic Stress Disorde""",314
"""Somatoform Pain Disorde""",1


In [ ]:
df = df.with_columns(
    col("condition").str.replace(r"^(.+(?:rd|ev)e)$", "${1}r").alias("condition")
)

## Decoding HTML entities in `review`s

It turns out that many HTML entities besides `&#039;` are present in the `review` column
and using `html.unescape()` as recommended by the Hugging Face tutorial
is probably more robust than replacing them individually. So we do just that.

In [ ]:
q = (
    df.select(col("review").str.extract_all(r"&#?[[:alnum:]]+;"))
    .explode("review")
    .get_column("review")
    .value_counts()
    .sort("count", descending=True)
); len(q)

45

In [ ]:
q.head(10)

review,count
str,u32
"""&#039;""",262415
null,55984
"""&quot;""",21262
"""&amp;""",13047
"""&rsquo;""",3029
"""&gt;""",162
"""&rdquo;""",117
"""&ldquo;""",116
"""&eacute;""",111


In [ ]:
import html

df = df.with_columns(col("review").map_elements(html.unescape, return_dtype=pl.String))

## Simplifying newlines in `review`s

Another issue of the `review`s pointed out by the Hugging Face tutorial is
the presence of many different kinds of newline characters.
I decide to replace all consecutive newline characters with one `\n`.

In [ ]:
df.filter(col("uniqueID") == 121260).item(0, "review")

'"More than Depression I take Effexor for Anxiety and it works well.  The only problem\r\r\noccurs when I forget to take it.  Within in hours I experience withdrawal symptoms\r\r\nsuch as light headiness and an occasional brief buzzing in my head."'

In [ ]:
df = df.with_columns(col("review").str.replace_all(r"[\r\n]+", "\n"))

In [ ]:
df.filter(col("uniqueID") == 121260).item(0, "review")

'"More than Depression I take Effexor for Anxiety and it works well.  The only problem\noccurs when I forget to take it.  Within in hours I experience withdrawal symptoms\nsuch as light headiness and an occasional brief buzzing in my head."'

## Reformatting `date`

Finally, I reformat the `dates` into "yyyy-mm-dd", which is sortable as string.

In [ ]:
df = df.with_columns(col("date").str.to_date("%-d-%b-%y"))

## Saving and sharing

The test set is processed the same way and the procedure is omitted for brevity.

In [ ]:
if DATA_DIR:
    import os

    os.makedirs(DATA_DIR, exist_ok=True)
    df.write_csv(f"{DATA_DIR}/train.tsv", separator="\t")

if REPO_ID and SECRET:
    !uv pip install --system -q datasets

    from datasets import load_dataset
    from google.colab import userdata

    dataset = load_dataset(
        "csv",
        data_files=dict(train=f"{DATA_DIR}/train.tsv", test=f"{DATA_DIR}/test.tsv"),
        delimiter="\t",
    )

    dataset.push_to_hub(REPO_ID, token=userdata.get(SECRET))

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/162 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/54 [00:00<?, ?ba/s]

[1]: https://pola.rs/
[2]: https://huggingface.co/learn/nlp-course/chapter5/3
[3]: https://www.kaggle.com/datasets/jessicali9530/kuc-hackathon-winter-2018
[4]: https://huggingface.co/datasets/dd-n-kk/uci-drug-review-cleaned